In [1]:
from pyspark.sql import functions as f

from pyspark.sql.functions  import col


#from ut_base import ut_base as ut_base

from ut_spark import ut_spark
from ut_store import ut_store
from ut_log import ut_log
from ut_base import ut_base

class CustomError(Exception):
    pass

spark = ut_spark.get_sparksession()

pipeline_log_path:


####  Read data from raw layer

In [2]:

#read test data from law layer
df= spark.read.format("csv")\
.option("header", "true")\
.option("path", "C:/demo/capstone/raw/2000_test.csv")\
.load()


In [3]:
print("count: {}".format(df.count()))


count: 10


In [4]:
df.toPandas().head(30)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2000,1,28,5,1647,1647,1906,1859,HP,154,...,15,11,0,NA,0,NA,NA,NA,NA,NA
1,2000,1,29,6,1648,1647,1939,1859,HP,154,...,5,47,0,NA,0,NA,NA,NA,NA,NA
2,2000,1,30,7,NA,1647,NA,1859,HP,154,...,0,0,1,NA,0,NA,NA,NA,NA,NA
3,2000,1,31,1,1645,1647,1852,1859,HP,154,...,7,14,0,NA,0,NA,NA,NA,NA,NA
4,2000,1,1,6,842,846,1057,1101,HP,609,...,3,8,0,NA,0,NA,NA,NA,NA,NA
5,2000,1,2,7,849,846,1148,1101,HP,609,...,8,24,0,NA,0,NA,NA,NA,NA,NA
6,2000,1,3,1,844,846,1121,1101,HP,609,...,6,27,0,NA,0,NA,NA,NA,NA,NA
7,2000,1,1,6,1702,1657,1912,1908,HP,611,...,5,13,0,NA,0,NA,NA,NA,NA,NA
8,2000,1,2,7,1658,1657,1901,1908,HP,611,...,3,7,0,NA,0,NA,NA,NA,NA,NA
9,2000,1,3,1,1656,1657,1922,1908,HP,611,...,5,20,0,NA,0,NA,NA,NA,NA,NA


#### Check data from processed layer

In [6]:
fp='C:/demo/testaota/processed/flight'
df = spark.read.parquet(fp)
df.toPandas().head(20)

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,row_id1,year
0,1,28,5,1647,1647,1906,1859,HP,154,N808AW,233,7,0,ATL,PHX,1587,0,2000
1,1,29,6,1648,1647,1939,1859,HP,154,N653AW,239,40,1,ATL,PHX,1587,1,2000
2,1,31,1,1645,1647,1852,1859,HP,154,N806AW,226,-7,-2,ATL,PHX,1587,2,2000
3,1,1,6,842,846,1057,1101,HP,609,N158AW,244,-4,-4,ATL,PHX,1587,3,2000
4,1,2,7,849,846,1148,1101,HP,609,N656AW,267,47,3,ATL,PHX,1587,4,2000
5,1,3,1,844,846,1121,1101,HP,609,N803AW,244,20,-2,ATL,PHX,1587,5,2000
6,1,1,6,1702,1657,1912,1908,HP,611,N652AW,232,4,5,ATL,PHX,1587,6,2000
7,1,2,7,1658,1657,1901,1908,HP,611,N807AW,233,-7,1,ATL,PHX,1587,7,2000
8,1,3,1,1656,1657,1922,1908,HP,611,N807AW,241,14,-1,ATL,PHX,1587,8,2000


#### Check data in integration_layer

#### check  table dim_date

In [7]:
fp='C:/demo/testaota/integrated/dim_date'

In [8]:
df = spark.read.parquet(fp)
df.show()

+--------+----+-----+----------+---------+
| date_id|Year|Month|DayofMonth|DayOfWeek|
+--------+----+-----+----------+---------+
|20000128|2000|    1|        28|        5|
|20000129|2000|    1|        29|        6|
|20000131|2000|    1|        31|        1|
|20000101|2000|    1|         1|        6|
|20000102|2000|    1|         2|        7|
|20000103|2000|    1|         3|        1|
+--------+----+-----+----------+---------+



#### Check table  dim_origin_dest

In [2]:
fp='C:/demo/testaota/integrated/dim_origin_dest'


In [3]:
df= spark.read.parquet(fp)
df.toPandas().head(10)

,orig_dest_id,origin,dest,distance,orig_airport,orig_city,orig_state,orig_country,dest_airport,dest_city,dest_state,dest_country
0,PHX_ATL,ATL,PHX,1587,William B Hartsfield-Atlanta Intl,Atlanta,GA,USA,Phoenix Sky Harbor International,Phoenix,AZ,USA


#### Check fact table

In [11]:
r_fp ='C:/demo/testaota/integrated/fact_flight'

In [12]:
df_ff =df_pl = spark.read.parquet(r_fp)

In [13]:
df_ff.toPandas().head(30)

,month,seq_id,date_id,orig_dest_id,flightNum,carrier_id,plane_id,DepTime,scheduled_deptime,ArrTime,scheduled_arrtime,ArrDelay,year
0,1,4,20000102,PHX_ATL,609,HP,N656AW,849,846,1148,1101,47,2000
1,1,5,20000103,PHX_ATL,609,HP,N803AW,844,846,1121,1101,20,2000
2,1,8,20000103,PHX_ATL,611,HP,N807AW,1656,1657,1922,1908,14,2000
3,1,0,20000128,PHX_ATL,154,HP,N808AW,1647,1647,1906,1859,7,2000
4,1,1,20000129,PHX_ATL,154,HP,N653AW,1648,1647,1939,1859,40,2000
5,1,6,20000101,PHX_ATL,611,HP,N652AW,1702,1657,1912,1908,4,2000
6,1,2,20000131,PHX_ATL,154,HP,N806AW,1645,1647,1852,1859,-7,2000
7,1,3,20000101,PHX_ATL,609,HP,N158AW,842,846,1057,1101,-4,2000
8,1,7,20000102,PHX_ATL,611,HP,N807AW,1658,1657,1901,1908,-7,2000
